# Echo-B 1B4 (Basemodel Training)
This model is a custom 1.4B model containing
- 96 layers
- 1024 embedding size

It was initialized using the original RWKV trainer here : https://github.com/PicoCreator/RWKV-LM-LoRA/blob/picocreator-init-memory-experiment/notebook/echo-B-1B4-training.ipynb

The goal of this model training is to exceed RWKV model memory, with its increased layer size

While the original enwiki-v1 replication of 1.5B did not fully match the finetuned version memory capacity, it managed to match raven 1.5B performance levels.
This helps prove that the training method used worked, and does not require training a full "pile+" model from scratch

As such adjustments were made in the training process (based on learnings in the original), and applied to this iteration. Where we will be training a 96 layer varient from scratch

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

# Basic Setup

In [1]:
# # First lets get the blank init model, these init model was generated
# # using the original RWKV-LM repo (as at this point of writing, this repo cannot init a model)
# #
# # As such I have preinitialized these blank models and uploaded them to HF for convinence
# #
# # You can also download the model for each stage respectively
# #
# # Comment back in this cell if you need it
# !mkdir -p ../../../model/
# !mkdir -p ../../../datapath/
# !mkdir -p ../../../checkpoint/
# !rm -rf ../../../model/Echo-B-1B4-Init.pth
# !cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-B-1B4-Init.pth
# !ls -alh ../../../model/Echo-B-1B4-Init.pth

# # Stage specific model download
# !cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-B-1B4-Stage1.pth
# !cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-B-1B4-Stage2.pth

In [2]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="Echo-B-1B4"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /root/picocreator-memory-experiment/notebook/experiment/memory-enwiki-v2
TRAINER_DIR: /root/picocreator-memory-experiment/RWKV-v4neo
PROJECT_DIR: /root/picocreator-memory-experiment


## Stage 1 : Foundation model training

In [3]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/Echo-B-1B4-enwiki.yaml"

Found cached dataset parquet (/root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 69.64it/s]
Loading cached processed dataset at /root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-197c10b1cc695da5_*_of_00064.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-3a57548a71476a57_*_of_00064.arrow


In [4]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/Echo-B-1B4-enwiki.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki Foundation (ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 1053079484
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 1053079484
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230713_163624-p1orui5k
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run (8x3090) Echo-B-1B4 - Enwiki Foundation (ctx=4096, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Memory-Experiment
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Memory-Experiment/runs/p1orui5k
Using /root/.cache/torch_extensions/py311_cu118 as

In [19]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/Echo-B-1B4-enwiki/last.ckpt" "../model/Echo-B-1B4-Stage1.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/Echo-B-1B4-Stage1.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/Echo-B-1B4-enwiki/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 1734 params 1412675584 elements
Saving fp32 state dict to ../model/Echo-B-1B4-Stage1.pth
-rw-r--r-- 1 root root 5.3G Jul 14 03:17 ../model/Echo-B-1B4-Stage1.pth


In [8]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && python3 dragon_test.py ../model/Echo-B-1B4-Stage1.pth "cuda fp32"

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading ../model/Echo-B-1B4-Stage1.pth ...
Strategy: (total 96+1=97 layers)
* cuda [float32, float32], store 97 layers
0-cuda-float32-float32 1-cuda-float32-float32 2-cuda-float32-float32 3-cuda-float32-float32 4-cuda-float32-float32 5-cuda-float32-float32 6-cuda-float32-float32 7-cuda-float32-float32 8-cuda-float32-float32 9-cuda-float32-float32 10-cuda-float32-float32 11-cuda-float32-float32 12-cuda-float32-float32 13-cuda-float32-float32 14-cuda-float32-float32 15-cuda-float32-float32 16-cuda-float32-float32 17-cuda-float32-float32 18-cuda-float32-float32 19-cuda-float32-float32 20-cuda-float32-float32 21-cuda-float32-float32 22-cuda-float32-float32 23-cuda-float32-float32 24-cuda-float32-float32 25-cuda-float32-float32 26-cuda-float32-float32 27-cuda-float32-float32 28-cuda-float32-float32 29-cuda-float32-float32 30-cuda-float32-float32 31-cuda-float32-float32 32-cuda-float32-float32 33-cuda-float32-float32 34-cuda-float32-float32 35-cu

In [9]:
# Lets do a quick memory test
# (We dun expect this to work, as we have not finetune for memory recall, but its a baseline)
!python3 ./memory_script/eval_memory_guided.py "{PROJECT_DIR}/model/Echo-B-1B4-Stage1.pth"

Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_cuda...
RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 0

Loading /home/picocreator/rwkv-proj/picocreator-memory-experiment/model/Echo-B-1B4-Stage1.pth ...
Strategy: (total 96+1=97 layers)
* cuda [float32, float32], store 97 layers
0-cuda-float32-float32 1-cuda-float32-float32 2-cuda-float32-float32 3-cuda-float32-float32 4-cuda-float32-float32 5-cuda-float32-float32 6-cuda-float32-float32 7-cuda-float32-float32 8-cuda-float32-float32 9-cuda-float32-float32 10-cuda-float32-float32 11-cuda-float32-float32 12-cuda-float32-float32 13-cuda-float32-float32 14-cuda-flo

# Stage 2 : Instruct Tuning

In [8]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/Echo-B-1B4-instruct.yaml"

Extracting data files: 100%|████████████████████| 1/1 [00:00<00:00, 1532.45it/s]
Setting num_proc from 64 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 657.72it/s]


In [21]:
# Start the instruct finetuning
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/Echo-B-1B4-instruct.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Instruct (train-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 2683499085
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 2683499085
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230714_032628-nxeap7ls
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run (8x3090) Echo-B-1B4 - Instruct (train-ctx=4096, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Memory-Experiment
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Memory-Experiment/runs/nxeap7ls
Using /root/.cache/torch_extensions/py311_cu118 as Py

In [22]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/Echo-B-1B4-instruct/last.ckpt" "../model/Echo-B-1B4-Stage2.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/Echo-B-1B4-Stage2.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/Echo-B-1B4-instruct/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 1734 params 1412675584 elements
Saving fp32 state dict to ../model/Echo-B-1B4-Stage2.pth
-rw-r--r-- 1 root root 5.3G Jul 14 04:15 ../model/Echo-B-1B4-Stage2.pth


In [23]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && python3 dragon_test.py "../model/Echo-B-1B4-Stage2.pth" "cuda fp32"

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading ../model/Echo-B-1B4-Stage2.pth ...
Strategy: (total 96+1=97 layers)
* cuda [float32, float32], store 97 layers
0-cuda-float32-float32 1-cuda-float32-float32 2-cuda-float32-float32 3-cuda-float32-float32 4-cuda-float32-float32 5-cuda-float32-float32 6-cuda-float32-float32 7-cuda-float32-float32 8-cuda-float32-float32 9-cuda-float32-float32 10-cuda-float32-float32 11-cuda-float32-float32 12-cuda-float32-float32 13-cuda-float32-float32 14-cuda-float32-float32 15-cuda-float32-float32 16-cuda-float32-float32 17-cuda-float32-float32 18-cuda-float32-float32 19-cuda-float32-float32 20-cuda-float32-float32 21-cuda-float32-float32 22-cuda-float32-float32 23-cuda-float32-float32 24-cuda-float32-float32 25-cuda-float32-float32 26-cuda-float32-float32 27-cuda-float32-float32 28-cuda-float32-float32 29-cuda-float32-float32 30-cuda-float32-float32 31-cuda-float32-float32 32-cuda-float32-float32 33-cuda-float32-float32 34-cuda-float32-float32 35-cu

In [24]:
# Lets do a quick memory test
# (We dun expect this to work, as we have not finetune for memory recall, but its a baseline)
!python3 ./memory_script/eval_memory_guided.py "{PROJECT_DIR}/model/Echo-B-1B4-Stage2.pth"

Using /root/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu118/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_cuda...
RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 0

Loading /root/picocreator-memory-experiment/model/Echo-B-1B4-Stage2.pth ...
Strategy: (total 96+1=97 layers)
* cuda [float32, float32], store 97 layers
0-cuda-float32-float32 1-cuda-float32-float32 2-cuda-float32-float32 3-cuda-float32-float32 4-cuda-float32-float32 5-cuda-float32-float32 6-cuda-float32-float32 7-cuda-float32-float32 8-cuda-float32-float32 9-cuda-float32-float32 10-cuda-float32-float32 11-cuda-float32-float32 12-cuda-float32-float32 13-cuda-float32-float32 14-cuda-float32-float32 15-cuda-float32-float32 16-cuda-f